In [1]:
import numpy as np                                  # type: ignore
from skimage.measure import label, regionprops      # type: ignore
from skimage.io import imread, imshow               # type: ignore
from skimage.filters import threshold_yen           # type: ignore
import glob
from dataclasses import dataclass
import napari
import os
import time
import myfunctions as mf
from tqdm import tqdm

In [2]:
sequence = mf.read_4Dsequence('P28A_FT_H_Exp1', first_slice=100, last_slice=104+1, OS='Windows')

In [9]:
print(sequence[0,0,:,:].nbytes)
copy = np.zeros_like(sequence[0,0,:,:], dtype=np.ushort)
print(copy.nbytes*4)

3276800
3276800
